In [ ]:
#pip install entsoe-py

Prices are in MW (Mega Watt), which equals to 1000 kW or 1.000.000 W. Therefore the generated table gets divided by 1.000.000 to get the price for one W.

# Testing the individual steps

In [333]:
date = "2023-03-09"
timezone = 'Europe/Brussels'

In [334]:
import pandas as pd
from datetime import datetime, timedelta
from entsoe import EntsoePandasClient
import pytz
date = pd.to_datetime(date,format= '%Y-%m-%d')
# looking for tommorow prices
date = date + timedelta(days = 1)
current_timezone = pytz.timezone(timezone)
date = current_timezone.localize(date)
start = (date - timedelta(days= 20)).normalize()
end = (date + timedelta(days = 20)).normalize()
country_code = 'DE_LU'
client = EntsoePandasClient(api_key='6f67ccf4-edb3-4100-a850-969c73688627')
df = client.query_day_ahead_prices(country_code = country_code, start = start, end = end)
df

2023-02-18 00:00:00+01:00     28.14
2023-02-18 01:00:00+01:00     36.44
2023-02-18 02:00:00+01:00     35.82
2023-02-18 03:00:00+01:00     44.41
2023-02-18 04:00:00+01:00     62.98
                              ...  
2023-03-07 19:00:00+01:00    143.91
2023-03-07 20:00:00+01:00    152.14
2023-03-07 21:00:00+01:00    136.43
2023-03-07 22:00:00+01:00    122.21
2023-03-07 23:00:00+01:00    115.49
Freq: 60T, Length: 432, dtype: float64

In [335]:
# handling problem with missing price data for more than 24 hours ahead
indicator = date.replace(hour=0, minute=0, second=0, microsecond=0)
if(indicator < max(df.index).replace(hour=0, minute=0, second=0, microsecond=0)):
    range_hours = pd.date_range(start=date, freq="H", periods=48)
    df = df.loc[range_hours]
if(indicator.strftime('%Y-%m-%d') == max(df.index).strftime('%Y-%m-%d')):
    date_48 = date + timedelta(days=1)
    for hour in range(24):
        dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
        # Get the price from the day before at this hour
        day_before = dt - timedelta(days=1)
        value = df.loc[day_before]
        # Append the new row to the series
        df.loc[dt] = value
    range_hours = pd.date_range(start=date, freq="H", periods=48)
    df = df.loc[range_hours]
if(indicator > max(df.index).replace(hour=0, minute=0, second=0, microsecond=0)):
    date_now = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    date_48 = date_now + timedelta(days=1)
    for hour in range(24):
        dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
        dt = current_timezone.localize(dt)
        # Get the price from the day before at this hour
        day_before = date_now.replace(hour=hour, minute=0, second=0, microsecond=0)
        day_before = current_timezone.localize(day_before)
        value = df.loc[day_before]
        # Append the new row to the series
        df.loc[dt] = value
    date_now = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    date_48 = date_now + timedelta(days=2)
    for hour in range(24):
        dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
        dt = current_timezone.localize(dt)
        # Get the price from the day before at this hour
        day_before = date_now.replace(hour=hour, minute=0, second=0, microsecond=0)
        day_before = current_timezone.localize(day_before)
        value = df.loc[day_before]
        # Append the new row to the series
        df.loc[dt] = value
    date_tomorrow = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
    date_tomorrow = current_timezone.localize(date_tomorrow)
    range_hours = pd.date_range(start=date_tomorrow, freq="H", periods=48)
    df = df.loc[range_hours]

C:\Users\chris\AppData\Local\Temp\ipykernel_804\313332653.py:39: FutureWarning: Setitem-like behavior with mismatched timezones is deprecated and will change in a future version. Instead of raising (or for Index, Series, and DataFrame methods, coercing to object dtype), the value being set (or passed as a fill_value, or inserted) will be cast to the existing DatetimeArray/DatetimeIndex/Series/DataFrame column's timezone. To retain the old behavior, explicitly cast to object dtype before the operation.
  df.loc[dt] = value


In [336]:
df

2023-03-07 00:00:00+01:00    136.10
2023-03-07 01:00:00+01:00    125.09
2023-03-07 02:00:00+01:00    123.31
2023-03-07 03:00:00+01:00    123.12
2023-03-07 04:00:00+01:00    123.75
2023-03-07 05:00:00+01:00    137.44
2023-03-07 06:00:00+01:00    159.80
2023-03-07 07:00:00+01:00    185.00
2023-03-07 08:00:00+01:00    217.09
2023-03-07 09:00:00+01:00    188.35
2023-03-07 10:00:00+01:00    170.26
2023-03-07 11:00:00+01:00    160.84
2023-03-07 12:00:00+01:00    149.92
2023-03-07 13:00:00+01:00    144.95
2023-03-07 14:00:00+01:00    144.30
2023-03-07 15:00:00+01:00    145.78
2023-03-07 16:00:00+01:00    152.00
2023-03-07 17:00:00+01:00    162.01
2023-03-07 18:00:00+01:00    184.86
2023-03-07 19:00:00+01:00    174.61
2023-03-07 20:00:00+01:00    147.95
2023-03-07 21:00:00+01:00    130.39
2023-03-07 22:00:00+01:00    120.65
2023-03-07 23:00:00+01:00    108.58
2023-03-08 00:00:00+01:00    136.10
2023-03-08 01:00:00+01:00    125.09
2023-03-08 02:00:00+01:00    123.31
2023-03-08 03:00:00+01:00   

# Whole price agent

In [305]:
###################################################################################################
# price agent #####################################################################################
###################################################################################################
class Price_Agent(): 

    def return_day_ahead_prices(self, date, timezone = 'Europe/Brussels'):
        import pandas as pd
        from datetime import datetime, timedelta
        from entsoe import EntsoePandasClient
        import pytz
        date = pd.to_datetime(date,format= '%Y-%m-%d')
        # looking for tommorow prices
        date = date + timedelta(days = 1)
        current_timezone = pytz.timezone(timezone)
        date = current_timezone.localize(date)
        start = (date - timedelta(days= 20)).normalize()
        end = (date + timedelta(days = 20)).normalize()
        country_code = 'DE_LU'
        client = EntsoePandasClient(api_key='6f67ccf4-edb3-4100-a850-969c73688627')
        df = client.query_day_ahead_prices(country_code = country_code, start = start, end = end)
        
        # handling problem with missing price data for more than 24 hours ahead
        indicator = date.replace(hour=0, minute=0, second=0, microsecond=0)
        if(indicator < max(df.index).replace(hour=0, minute=0, second=0, microsecond=0)):
            range_hours = pd.date_range(start=date, freq="H", periods=48)
            df = df.loc[range_hours]
        if(indicator.strftime('%Y-%m-%d') == max(df.index).strftime('%Y-%m-%d')):
            date_48 = date + timedelta(days=1)
            for hour in range(24):
                dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
                # Get the price from the day before at this hour
                day_before = dt - timedelta(days=1)
                value = df.loc[day_before]
                # Append the new row to the series
                df.loc[dt] = value
            range_hours = pd.date_range(start=date, freq="H", periods=48)
            df = df.loc[range_hours]
        if(indicator > max(df.index).replace(hour=0, minute=0, second=0, microsecond=0)):
            date_now = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
            date_48 = date_now + timedelta(days=1)
            for hour in range(24):
                dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
                dt = current_timezone.localize(dt)
                # Get the price from the day before at this hour
                day_before = date_now.replace(hour=hour, minute=0, second=0, microsecond=0)
                day_before = current_timezone.localize(day_before)
                value = df.loc[day_before]
                # Append the new row to the series
                df.loc[dt] = value
            date_now = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
            date_48 = date_now + timedelta(days=2)
            for hour in range(24):
                dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
                dt = current_timezone.localize(dt)
                # Get the price from the day before at this hour
                day_before = date_now.replace(hour=hour, minute=0, second=0, microsecond=0)
                day_before = current_timezone.localize(day_before)
                value = df.loc[day_before]
                # Append the new row to the series
                df.loc[dt] = value
            date_tomorrow = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
            date_tomorrow = current_timezone.localize(date_tomorrow)
            range_hours = pd.date_range(start=date_tomorrow, freq="H", periods=48)
            df = df.loc[range_hours]
        return df

# Testing the Agent

In [306]:
date = "2023-03-04"

In [307]:
PA = Price_Agent()
price_df = PA.return_day_ahead_prices(date)

In [308]:
price_df

2023-03-05 00:00:00+01:00    107.53
2023-03-05 01:00:00+01:00    106.25
2023-03-05 02:00:00+01:00    105.55
2023-03-05 03:00:00+01:00    106.84
2023-03-05 04:00:00+01:00    104.99
2023-03-05 05:00:00+01:00    105.81
2023-03-05 06:00:00+01:00    105.94
2023-03-05 07:00:00+01:00    108.39
2023-03-05 08:00:00+01:00    107.72
2023-03-05 09:00:00+01:00    107.73
2023-03-05 10:00:00+01:00    105.18
2023-03-05 11:00:00+01:00    107.55
2023-03-05 12:00:00+01:00    110.25
2023-03-05 13:00:00+01:00    107.73
2023-03-05 14:00:00+01:00    110.22
2023-03-05 15:00:00+01:00    115.14
2023-03-05 16:00:00+01:00    129.37
2023-03-05 17:00:00+01:00    145.14
2023-03-05 18:00:00+01:00    166.67
2023-03-05 19:00:00+01:00    172.37
2023-03-05 20:00:00+01:00    160.23
2023-03-05 21:00:00+01:00    152.49
2023-03-05 22:00:00+01:00    143.66
2023-03-05 23:00:00+01:00    130.13
2023-03-06 00:00:00+01:00    136.10
2023-03-06 01:00:00+01:00    125.09
2023-03-06 02:00:00+01:00    123.31
2023-03-06 03:00:00+01:00   